In [ ]:
import gradio as gr
import hopsworks
import joblib
import pandas as pd
import os
import openmeteo_requests
import requests_cache
from retry_requests import retry
from huggingface_hub import InferenceClient

# Set Hugging Face Token (if not in environment variables)
if "HF_TOKEN" not in os.environ:
    os.environ["HF_TOKEN"] = "HF_token"

# ==========================================
#  Model Part: Fetch Weather & Predict Wind
# ==========================================
def get_weather_forecast():
    # Fetch 7-day forecast (Same logic as File 3)
    cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    params = {
        "latitude": 39.4532, # Flores
        "longitude": -31.1274,
        "daily": ["temperature_2m_max", "precipitation_sum", "wind_gusts_10m_max", "wind_direction_10m_dominant"],
        "timezone": "Atlantic/Azores",
        "forecast_days": 7
    }
    
    responses = openmeteo.weather_api("https://api.open-meteo.com/v1/forecast", params=params)
    response = responses[0]
    daily = response.Daily()
    
    df = pd.DataFrame({
        "date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
        ),
        "temperature_max": daily.Variables(0).ValuesAsNumpy(),
        "precipitation": daily.Variables(1).ValuesAsNumpy(),
        "wind_gusts": daily.Variables(2).ValuesAsNumpy(),
        "wind_direction": daily.Variables(3).ValuesAsNumpy(),
    })
    df['date_str'] = df['date'].dt.strftime('%Y-%m-%d')
    return df

def get_prediction_summary():
    """
    Run the ML model and return a text summary of the 7-day wind forecast.
    """
    print(" Connecting to Hopsworks to download model...")
    try:
        project = hopsworks.login()
        mr = project.get_model_registry()
        # Note: Loading the new wind model
        model = mr.get_model(name="azores_wind_model", version=1)
        model_dir = model.download()
        
        model_path = os.path.join(model_dir, "azores_wind_model.pkl")
        trained_model = joblib.load(model_path)
        
        df = get_weather_forecast()
        features = df[['temperature_max', 'precipitation', 'wind_gusts', 'wind_direction']]
        
        preds = trained_model.predict(features)
        
        summary = ""
        for date, wind, gust in zip(df['date_str'], preds, df['wind_gusts']):
            wind_kmh = max(0, wind)
            summary += f"- {date}: Predicted Wind {wind_kmh:.1f} km/h (Gusts {gust:.1f} km/h)\n"
        
        return summary
    except Exception as e:
        return f"Failed to fetch prediction data: {str(e)}"

# Pre-load data during initialization
print("Initialization: Fetching latest data and model...")
CACHE_FORECAST = get_prediction_summary()
print("Data ready!")

# ==========================================
# LLM Part: Define Persona and Logic
# ==========================================
def chatbot_response(message, history):
    # Initialize HF Client
    client = InferenceClient(
        "Qwen/Qwen2.5-7B-Instruct", 
        token=os.environ["HF_TOKEN"]
    )
    
    # System Prompt - Define persona as 'Captain Joao'
    system_prompt = f"""
    You are 'Captain Joao', an experienced and humorous speedboat captain in Flores, Azores.
    Your job is to take tourists to the neighboring Corvo Island.
    
    Here is the REAL wind forecast for the next 7 days (based on ML predictions):
    {CACHE_FORECAST}
    
    **Rules:**
    1. Answer based on the data above.
    2. If predicted wind > 30 km/h: Be apologetic and warn that the boat is cancelled due to high waves. Suggest wine instead.
    3. If predicted wind < 30 km/h: Be cheerful and say it's a perfect day for sailing!
    4. Keep answers short, nautical, and use emojis like 🌊, 🚤, ⚓, 🌬️.
    """

    messages = []
    messages.append({"role": "system", "content": system_prompt})
    
    # Append history
    messages.extend(history)
    messages.append({"role": "user", "content": message})

    # Stream the response
    partial_message = ""
    try:
        for token in client.chat_completion(messages, max_tokens=500, stream=True):
            if token.choices[0].delta.content:
                partial_message += token.choices[0].delta.content
                yield partial_message
    except Exception as e:
        yield f" Error: {str(e)}"

# ==========================================
#  Gradio Interface
# ==========================================
demo = gr.ChatInterface(
    fn=chatbot_response,
    title="🚤 Flores-Corvo Boat Forecaster",
    description="Architecture: Gradient Boosting Wind Prediction + LLM (Qwen-2.5)",
    examples=[
        "Will the boat run tomorrow?",
        "Is the weather good for sailing this weekend?",
        "What if the wind is too strong?",
    ],
    cache_examples=False
)

if __name__ == "__main__":
    demo.launch(share=True, debug=True)

⏳ Initialization: Fetching latest data and model...
🤖 Connecting to Hopsworks to download model...
2026-01-13 22:31:06,430 INFO: Initializing external client
2026-01-13 22:31:06,430 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-13 22:31:07,071 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-13 22:31:08,071 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1303706


Downloading: 0.000%|          | 0/203984 elapsed<00:00 remaining<?

2026-01-13 22:31:11,097 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-13 22:31:11,097 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

2026-01-13 22:31:11,098 WARNING: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

✅ Data ready!
* Running on local URL:  http://127.0.0.1:7861
2026-01-13 22:31:11,302 INFO: HTTP Request: GET http://127.0.0.1:7861/gradio_api/startup-events "HTTP/1.1 200 OK"
2026-01-13 22:31:11,314 INFO: HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"
2026-01-13 22:31:11,332 INFO: HTTP Request: HEAD https://huggingface.co/api/telemetry/https%3A/api.gradio.app/gradio-initiated-analytics "HTTP/1.1 200 OK"
2026-01-13 22:31:11,849 INFO: HTTP Request: GET https://api.gradio.app

2026-01-13 22:31:13,861 INFO: HTTP Request: HEAD https://huggingface.co/api/telemetry/https%3A/api.gradio.app/gradio-launched-telemetry "HTTP/1.1 200 OK"
2026-01-13 22:31:35,112 WARNING: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.

2026-01-13 22:31:35,831 WARNING: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.

2026-01-13 22:31:36,122 INFO: HTTP Request: POST https://router.huggingface.co/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-13 22:31:52,697 WARNING: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.

2026-01-13 22:31:53,065 INFO: HTTP Request: POST https://router.huggingface.co/v1/chat/completions "HTTP/1.1 200 OK"
2026-01-13 22:31:53,969 WARNING: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.

2026-01-13 22:32:04,254 